In [1]:
import os
import sys
import numpy as np
from o2_utils.selectors import find_files_from_pattern

In [15]:
# slurm params
mem = "2GB"  # actually uses more like ~1GB, but 2 is safe
partition = "short"
compression_preset = "slow"
delete_original = True
# python_args = '{input_file} {output_file} --preset {compression_preset}'  # do this and specify the output file if you want to save the compressed videos somewhere separate
python_args = '{input_file} --preset {compression_preset}'  # this plus the flag --delete_original will replace the original file with the compressed one

In [16]:
sbatch_command = f"sbatch -p {partition} --mem={mem} --time={{time}} -o {{output_dir}}/{{vid_name}}_COMPRESSION_%j.out"
wrap_command = 'eval "$(conda shell.bash hook)"; conda activate dataPy_NWB2; module load ffmpeg; python /n/groups/datta/Jonah/20231121_6cam_sniff/safe_compress.py '
wrap_command = wrap_command + python_args
if delete_original:
    wrap_command += " --delete_original"

if compression_preset == "slow":
    time = "10:00:00"  # side4 vid failed in test run at 10 hrs, all others took 6-8 hrs

In [17]:
wrap_command

'eval "$(conda shell.bash hook)"; conda activate dataPy_NWB2; module load ffmpeg; python /n/groups/datta/Jonah/20231121_6cam_sniff/safe_compress.py {input_file} --preset {compression_preset} --delete_original'

In [18]:
base_path = "/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/"
# mice = ["J01701", "J01702", "J01703"]
mice = ["J04301", "J04302", "J04303", "J04304"]
for mouse in mice:
    sessions = os.listdir(os.path.join(base_path, mouse))
    for session in sessions:
        videos = find_files_from_pattern(os.path.join(base_path, mouse, session), "*.mp4", exclude_patterns=["COMPRESSED", "azure"], n_expected=6)

        for vid in videos:
            output_dir = os.path.dirname(vid)
            vid_name = os.path.basename(vid)
            _name, ext = os.path.splitext(vid_name)
            output_vid = os.path.join(output_dir, _name + ".COMPRESSED" + ".mp4")
            formatted_sbatch_command = sbatch_command.format(output_dir=output_dir, vid_name=vid_name, time=time)
            formatted_wrap_command = wrap_command.format(input_file=vid, compression_preset=compression_preset)
            command = f"{formatted_sbatch_command} --wrap \'{formatted_wrap_command}\'"
            print(command)


sbatch -p short --mem=2GB --time=10:00:00 -o /n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04301/20240208_J04301/20240208_J04301.side3.mp4_COMPRESSION_%j.out --wrap 'eval "$(conda shell.bash hook)"; conda activate dataPy_NWB2; module load ffmpeg; python /n/groups/datta/Jonah/20231121_6cam_sniff/safe_compress.py /n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04301/20240208_J04301/20240208_J04301.side3.mp4 --preset slow --delete_original'
sbatch -p short --mem=2GB --time=10:00:00 -o /n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04301/20240212_J04301/20240212_J04301.side2.mp4_COMPRESSION_%j.out --wrap 'eval "$(conda shell.bash hook)"; conda activate dataPy_NWB2; module load ffmpeg; python /n/groups/datta/Jonah/20231121_6cam_sniff/safe_compress.py /n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04301/20240212_J04301/20240212_J04301.side2.mp4 --preset slow --delete_original'
sbatch -p short --mem=2GB --time=10:00:00 -o /n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/